# Time-inVariant/Varying Multi-Variate Autoregressive Process Simulator/Estimator
### TV/TiV MVAR sim/est

In [ ]:
def TiV_MVAR_simulator(A, x0, t, noise=None):
    """
    Simulate a time-invariant multivariate autoregressive (MVAR) process.

    This function generates data from an MVAR model with fixed coefficients
    across time. The simulation allows optional input noise; if none is
    provided, Gaussian noise is used by default.

    Parameters
    ----------
    A : ndarray, shape (m, m, lags)
        MVAR coefficient matrices for each lag, where `m` is the number of
        variables and `lags` is the model order.
    x0 : ndarray, shape (m, lags)
        Initial conditions for the simulation. Each column corresponds to the
        state of all variables at a lag time.
    t : ndarray, shape (n,)
        Array of time points. Determines the length of the simulated data.
    noise : ndarray, shape (m, n), optional
        Additive noise to the system. If None, standard normal Gaussian
        noise is generated.

    Returns
    -------
    x : ndarray, shape (m, n)
        Simulated MVAR data, where each row corresponds to a variable and each
        column to a time point.
    """

    import numpy as np

    n = t.shape[0]
    m = x0.shape[0]
    print(n)
    Lag = x0.shape[1]

    if noise is None:
        noise = np.random.normal(loc=0, scale=1, size=(m, n))

    x = np.zeros([m, n])
    x[:, :Lag] = x0

    for t_i in range(Lag, n):
        x[:, t_i] = noise[:, t_i]

        for p in range(Lag):
            x[:, t_i] += A[:, :, p].dot(x[:, t_i-p-1])


    return x


In [ ]:
def TiV_MVAR_estimator(data, MaxLagOrder, Order_selection_criterion=None, Order_selection_print=False):
    """
    Estimate a time-invariant multivariate autoregressive (MVAR) model from data.

    This function fits an MVAR model to multivariate time series data using
    the specified maximum lag order. It supports automatic lag order selection
    based on common information criteria.

    Parameters
    ----------
    data : ndarray, shape (m, n)
        Multivariate time series data, where `m` is the number of variables
        (channels) and `n` is the number of time points.
    MaxLagOrder : int
        Maximum lag order to consider when fitting the MVAR model.
    Order_selection_criterion : str, optional
        Criterion for automatic lag order selection. Can be 'aic', 'bic', 'hqic',
        or None to skip automatic selection. Default is None.
    Order_selection_print : bool, optional
        If True, prints the selected lag order during automatic selection.
        Default is False.

    Returns
    -------
    A_est : ndarray, shape (m, m, lags)
        Estimated MVAR coefficient matrices. The third dimension corresponds
        to the lag order.
    """

    import numpy as np
    import statsmodels.tsa.api as sm

    model = sm.VAR(data.T)

    Order = MaxLagOrder
    if Order_selection_criterion is not None:
        if Order_selection_print:
            print(f"selected order = {model.select_order(MaxLagOrder).selected_orders}")
        Order = model.select_order(MaxLagOrder).selected_orders[Order_selection_criterion]

    if Order == 0: Order = 1

    results = model.fit(maxlags=Order, trend='c')

    A_est = results.coefs
    A_est = np.transpose(A_est, (1, 2, 0))  # Transpose to get shape (m, m, lags)


    return A_est

In [ ]:
def TV_MVAR_simulator(A, t_seg_edges, x0, t, noise=None):
    """
    Simulate a time-varying multivariate autoregressive (TV-MVAR) process.

    This function generates multivariate time series data from a TV-MVAR model,
    where the coefficient matrices can change at specified time segments. 
    Optional additive noise can be provided; otherwise, Gaussian noise is used.

    Parameters
    ----------
    A : list of ndarray, each with shape (m, m, lags)
        List of MVAR coefficient matrices for each segment. Each entry corresponds
        to a segment with its own lag-specific coefficients.
    t_seg_edges : list of int or float
        Time points (edges) where the MVAR coefficients change. The length
        of this list should match the number of segments in `A`.
    x0 : ndarray, shape (m, lags)
        Initial conditions for the simulation. Each column corresponds to the
        state of all variables at a lag time.
    t : ndarray, shape (n,)
        Array of time points. Determines the total length of the simulated data.
    noise : ndarray, shape (m, n), optional
        Additive noise for the system. If None, standard normal Gaussian noise
        is generated. Default is None.

    Returns
    -------
    x : ndarray, shape (m, n)
        Simulated TV-MVAR data, where each row corresponds to a variable
        and each column to a time point.
    """


    import numpy as np

    n = t.shape[0]
    m = x0.shape[0]
    Lag0 = x0.shape[1]

    if noise is None: noise = np.random.normal(loc=0, scale=1, size=(m, n))

    x = np.zeros([m, n])
    x[:, :Lag0] = x0

    seg_i = 0
    for t_i in range(Lag0, n):
        if t[t_i] >= t_seg_edges[seg_i]:   # ===== Updating A_i & lag when t changes =====
            A_seg = A[seg_i]
            Lag_seg = A_seg.shape[2]
            seg_i += 1
            
        x[:, t_i] = noise[:, t_i]
        for p in range(Lag_seg):
            x[:, t_i] += A_seg[:, :, p].dot(x[:, t_i-p-1])


    return x


In [ ]:
def Sliding_Window_Order_selection_func(
    data, MaxLagOrder, window_size, window_shift,
    Order_selection_criterion='aic', Order_selection_print=False
):
    """
    Estimate the order of a multivariate autoregressive (MVAR) model using a sliding window approach.

    This function applies a sliding window over multivariate time series data to
    determine the optimal MVAR model order within each window, based on a specified
    information criterion. The maximum order across all windows is returned.

    Parameters
    ----------
    data : ndarray, shape (m, n)
        Multivariate time series data, where `m` is the number of variables (channels)
        and `n` is the number of time points.
    MaxLagOrder : int
        Maximum lag order to consider for MVAR model fitting.
    window_size : int
        Number of time points in each sliding window.
    window_shift : int
        Step size to shift the sliding window.
    Order_selection_criterion : str, optional
        Criterion for automatic order selection in each window. Options are 'aic',
        'bic', 'hqic', or None to skip selection. Default is 'aic'.
    Order_selection_print : bool, optional
        If True, prints the selected order for each window. Default is False.

    Returns
    -------
    Order : int
        Estimated MVAR model order. It is the maximum order selected across all windows.
        If no order greater than zero is found, returns 1.
    """



    import statsmodels.tsa.api as sm

    m, n = data.shape
    Order_list = []

    Order = 0
    for start in range(0, n - window_size + 1, window_shift):
        end = start + window_size
        window_data = data[:, start:end]
        
        model = sm.VAR(window_data.T)
        window_Order = model.select_order(MaxLagOrder).selected_orders[Order_selection_criterion]
        Order_list += [window_Order]

        if Order_selection_print:
            print(f"selected order = {window_Order} for window [{start}, {end})")
        
        if window_Order>Order: Order = window_Order
    
    if Order == 0: Order = 1


    return Order


In [ ]:
def TV_MVAR_estimator_sliding_window(
    data, time, MaxLagOrder, window_size, window_shift,
    t_ref_loc='mid', Order_selection_criterion=None, Order_selection_print=False
):
    """
    Estimate a time-varying multivariate autoregressive (TV-MVAR) model using a sliding window.

    This function fits MVAR models to segments of multivariate time series data 
    defined by a sliding window. Coefficient matrices and their p-values are 
    returned for each window, providing a time-varying representation of the MVAR process.
    The reference time for each window can be chosen as the start, middle, or end of the window.

    Parameters
    ----------
    data : ndarray, shape (m, n)
        Multivariate time series data, where `m` is the number of variables (channels)
        and `n` is the number of time points.
    time : ndarray, shape (n,)
        Time points corresponding to each column of `data`.
    MaxLagOrder : int
        Maximum lag order for MVAR fitting.
    window_size : int
        Number of time points in each sliding window.
    window_shift : int
        Step size to shift the sliding window.
    t_ref_loc : str, optional
        Location of the reference time point for each window. Options are
        'mid' (default), 'start', or 'end'.
    Order_selection_criterion : str, optional
        Criterion for automatic order selection within each window. Can be
        'aic', 'bic', 'hqic', or None to skip order selection. Default is None.
    Order_selection_print : bool, optional
        If True, prints the selected order during order selection. Default is False.

    Returns
    -------
    A_est : list of ndarray, each shape (m, m, lags)
        List of estimated MVAR coefficient matrices for each window.
    t_est : list of float
        Reference time points corresponding to each estimated coefficient matrix.
    pValues : list of ndarray, each shape (m, m, lags)
        List of p-values associated with each coefficient matrix, indicating
        statistical significance of the estimated coefficients.
    """


    import numpy as np
    import statsmodels.tsa.api as sm

    m, n     = data.shape
    Order    = MaxLagOrder
    A_est    = []
    pValues  = []
    t_est    = []

    if Order_selection_criterion is not None:
        Order = Sliding_Window_Order_selection_func(data, MaxLagOrder, window_size, window_shift, Order_selection_criterion, Order_selection_print)

    for start in range(0, n - window_size + 1, window_shift):
        end = start + window_size
        window_data = data[:, start:end]

        model = sm.VAR(window_data.T)
        results = model.fit(maxlags=Order, trend='c')
        A_est_basket = results.coefs
        A_est_basket = np.transpose(A_est_basket, (1, 2, 0))

        A_est.append(A_est_basket)
        pValues.append(results.pvalues[1:,:].reshape([Order, m, m]).transpose((2, 1, 0)))

        if t_ref_loc == 'mid'  : t_est.append((time[start] + time[end-1]) / 2)
        if t_ref_loc == 'start': t_est.append(time[start])
        if t_ref_loc == 'end'  : t_est.append(time[end-1])


    return A_est, t_est, pValues


In [ ]:
def TV_MVAR_estimator_RLS(data, time, LagOrder=3, A0=None, P0=None, lambda_=0.90):
    """
    Estimate a time-varying multivariate autoregressive (TV-MVAR) model using the recursive least squares (RLS) algorithm.

    This function implements the RLS algorithm to estimate time-varying MVAR coefficients
    from multivariate time series data. It supports both fixed and adaptive forgetting factors
    and allows initialization of the coefficients and error covariance matrix.

    Parameters
    ----------
    data : ndarray, shape (m, n)
        Multivariate time series data, where `m` is the number of variables (channels)
        and `n` is the number of time points.
    time : ndarray, shape (n,)
        Time points corresponding to each column of `data`.
    LagOrder : int, optional
        Order of the MVAR model. Default is 3.
    A0 : ndarray, shape (m, m*LagOrder), optional
        Initial coefficient matrix for the RLS algorithm. Default is None (zeros).
    P0 : ndarray, shape (m*LagOrder, m*LagOrder), optional
        Initial covariance matrix for the RLS algorithm. Default is None (identity scaled by 1e5).
    lambda_ : float or list/tuple of two floats, optional
        Forgetting factor for the RLS algorithm. If a single float, a fixed forgetting factor
        is used. If a list or tuple of two floats, an adaptive forgetting factor is applied
        between the minimum and maximum values. Default is 0.90.

    Returns
    -------
    A_est : list of ndarray, each shape (m, m, LagOrder)
        Estimated MVAR coefficient matrices for each time point.
    t_est : list of float
        Time points corresponding to each estimated coefficient matrix.
    """

    import numpy as np

    m, n = data.shape

    adaptive_flag = False
    if isinstance(lambda_, (list, tuple)) and len(lambda_) == 2 and all(isinstance(x, (int, float)) for x in lambda_):
        adaptive_flag = True
        lambda_min = min(lambda_)
        lambda_max = max(lambda_)

    if A0 is not None:
        if A0.shape[0] != m or A0.shape[1] != m * LagOrder:
            raise ValueError(f"A0 must be of shape ({m}, {m * LagOrder})")

    if P0 is not None:
        if P0.shape[0] != m * LagOrder or P0.shape[1] != m * LagOrder:
            raise ValueError(f"P0 must be of shape ({m * LagOrder}, {m * LagOrder})")

    
    A = []; P = []
    for _ in range(LagOrder):
        A.append(np.zeros((m, m*LagOrder))  if A0 is None else A0)
        P.append(np.eye(m * LagOrder) * 1e5 if P0 is None else P0)
    phi = [[] for _ in range(LagOrder)]
    K   = [[] for _ in range(LagOrder)]
    e   = [[] for _ in range(LagOrder)]

    A_est = [A[-1].reshape([m, m, LagOrder], order='F')]
    t_est = [time[LagOrder-1]]
    PValues = [np.zeros((m, m, LagOrder))]


    for i in range(LagOrder, n, 1):
        # print(f"i = {i}")
        # ===== Computing vector of stacked past observations/lags =====
        phi_i = []
        for j in range(LagOrder):
            phi_i.append(data[:, i-j-1].reshape([-1, 1]))
        phi_i = np.concatenate(phi_i, axis=0)
        phi.append(phi_i)

        # ===== Adaptive forgetting factor for the first iteration =====
        if adaptive_flag:
            if i == LagOrder:
                lambda_ = lambda_max
            if i > LagOrder:
                e_norm = np.linalg.norm( e[i-1] )
                lambda_ = lambda_max - (lambda_max - lambda_min) * np.tanh(1e-3 * e_norm)

        # ===== Computing Kalman gain K =====
        K_i = P[i-1] @ phi[i] / (lambda_ + phi[i].T @ P[i-1] @ phi[i])
        K.append(K_i)

        # ===== Computing prediction error =====
        x_i = A[i-1] @ phi[i]
        e_i = data[:, i].reshape([-1,1]) - x_i
        e.append(e_i)

        # ===== Updating state estimate & error covariance =====
        A_i = A[i-1] + e[i] @ K[i].T
        A.append(A_i)

        P_i = (P[i-1] - K[i] @ phi[i].T @ P[i-1]) / lambda_
        # === Joseph-stabilized form:
        # I = np.eye(m * LagOrder)
        # P_i = (I - K[i] @ phi[i].T) @ P[i-1] @ (I - K[i] @ phi[i].T).T

        P.append(P_i)

        A_est.append(A[i].reshape([m, m, LagOrder], order='F'))
        t_est.append(time[i])


        # PValues_i = np.zeros((m, m * LagOrder))
        # for ch in range(m):
        #     A_ch = A[i][ch, :]
        #     sigma2 = e[i][ch]**2
        #     SE = np.sqrt(sigma2 * np.diag(P[i]))

        #     Z = A_ch / SE
        #     PValues_i[ch, :] = 2 * (1 - norm.cdf(abs(Z)))

        # PValues += [PValues_i.reshape([m, m, LagOrder], order='F')]



    return A_est, t_est
    # return A_est, t_est, PValues


In [ ]:
def TV_MVAR_estimator_KF(
    data, time, LagOrder, A0=None, P0=None, F=None, R=None, Q0=None,
    LAMBDA=None, smoothing=False
):
    """
    Estimate a time-varying multivariate autoregressive (TV-MVAR) model using the Kalman filter.

    This function applies the Kalman filter to estimate MVAR coefficients from
    multivariate time series data. It supports optional smoothing, customizable
    initial conditions, and process/measurement noise specifications.

    Parameters
    ----------
    data : ndarray, shape (m, n)
        Multivariate time series data, where `m` is the number of variables (channels)
        and `n` is the number of time points.
    time : ndarray, shape (n,)
        Time points corresponding to each column of `data`.
    LagOrder : int
        Order of the MVAR model.
    A0 : ndarray, shape (m, m*LagOrder), optional
        Initial coefficient matrix for the Kalman filter. Default is zeros.
    P0 : ndarray, shape (d, d), optional
        Initial covariance matrix for the state estimate, with d = m*m*LagOrder.
        Default is identity scaled by 1e5.
    F : ndarray, shape (d, d), optional
        State transition matrix. Default is identity.
    R : ndarray, shape (m, m), optional
        Measurement noise covariance matrix. Default is identity scaled by 1e2.
    Q0 : ndarray, shape (d, d), optional
        Initial process noise covariance matrix. Default is identity scaled by 1e-5.
    LAMBDA : ndarray, shape (d, d), optional
        Smoothing factors for the Kalman filter. Default is 0.5 * identity.
    smoothing : bool, optional
        If True, applies backward smoothing to refine coefficient estimates. Default is False.

    Returns
    -------
    A_est : list of ndarray, each shape (m, m, LagOrder)
        Estimated MVAR coefficient matrices for each time point. If `smoothing=True`,
        these are smoothed estimates.
    t_est : list of float
        Time points corresponding to each estimated coefficient matrix.
    """

    import numpy as np

    m, n = data.shape
    d = m*m*LagOrder

    if A0 is not None:
        if A0.shape[0] != m or A0.shape[1] != m * LagOrder:
            raise ValueError(f"A0 must be of shape ({m}, {m * LagOrder})")
    elif A0 is None: A0 = np.zeros([m, m * LagOrder])

    if P0 is not None:
        if P0.shape[0] != d or P0.shape[1] != d:
            raise ValueError(f"P0 must be of shape ({d}, {d})")
    elif P0 is None: P0 = np.eye(d) * 1e5

    if F is not None:
        if F.shape[0] != d or F.shape[1] != d:
            raise ValueError(f"F must be of shape ({d}, {d})")
    elif F is None: F = np.eye(d)

    if R is not None:
        if R.shape[0] != m or R.shape[1] != m:
            raise ValueError(f"R must be of shape ({m}, {m})")
    elif R is None: R = np.eye(m) * 1e2

    if LAMBDA is not None:
        if LAMBDA.shape[0] != d or LAMBDA.shape[1] != d:
            raise ValueError(f"LAMBDA must be of shape ({d}, {d})")
    elif LAMBDA is None: LAMBDA = np.eye(d) * 0.5

    if Q0 is not None:
        if Q0.shape[0] != d or Q0.shape[1] != d:
            raise ValueError(f"Q0 must be of shape ({d}, {d})")
    elif Q0 is None: Q0 = np.eye(d) * 1e-5


    GAMMA = []; P = []; Q = []
    for _ in range(LagOrder):
        GAMMA.append(A0.reshape([-1, 1], order='F'))
        P.append(P0)
        Q.append(Q0)
    Phi = [[] for _ in range(LagOrder)]
    H   = [[] for _ in range(LagOrder)]
    e   = [[] for _ in range(LagOrder)]
    S   = [[] for _ in range(LagOrder)]
    K   = [[] for _ in range(LagOrder)]

    A_est = [GAMMA[-1].reshape([m, m, LagOrder], order='F')]
    t_est = [time[LagOrder-1]]


    for i in range(LagOrder, n):
        # print(f"i={i}")
        # ===== Computing vector of stacked past observations/lags =====
        Phi_i = []
        for j in range(LagOrder):
            Phi_i.append(data[:, i-j-1].reshape([-1, 1]))
        Phi_i = np.concatenate(Phi_i, axis=0)
        Phi.append(Phi_i)

        H_i = np.kron(Phi[i].T, np.eye(m))
        H.append(H_i)

        # ===== Computing prediction error =====
        x_i = H[i] @ GAMMA[i-1]
        e_i = data[:, i].reshape([-1,1]) - x_i
        e.append(e_i)

        S_i = H[i] @ P[i-1] @ H[i].T + R
        S.append(S_i)

        # ===== Computing Kalman gain K =====
        K_i = P[i-1] @ H[i].T @ np.linalg.inv(S_i)
        K.append(K_i)

        # ===== Updating state estimate & error covariance =====
        GAMMA_i = F @ GAMMA[i-1] + K[i] @ e[i]
        GAMMA.append(GAMMA_i)

        Q_i = LAMBDA @ Q[i-1] + (np.eye(d) - LAMBDA) * np.linalg.norm(e[i])**2
        Q.append(Q_i)

        P_i = (np.eye(d) - K[i] @ H[i]) @ P[i-1] + Q[i]
        P.append(P_i)

        A_est.append(GAMMA[i].reshape([m, m, LagOrder], order='F'))
        t_est.append(time[i])


    if smoothing:
        # ===== Backward smoothing =====
        GAMMA_smooth = np.zeros(shape=[m*m*LagOrder, len(A_est)])

        for i in range(len(A_est)-3, -1, -1):
            GAMMA_i = A_est[i].reshape([-1, 1], order='F')
            W_i = P[i+LagOrder] @ np.linalg.inv( P[i+1+LagOrder] + Q[i+1+LagOrder] )
            GAMMA_smooth[:,i] = ( GAMMA_i + W_i @ (GAMMA_smooth[:,i+1].reshape([-1, 1]) - GAMMA_i) ).flatten()

        A_est_smooth = []
        for i in range(len(A_est)):
            A_est_smooth.append(GAMMA_smooth[:,i].reshape([m, m, LagOrder], order='F'))

        A_est = A_est_smooth
    
    return A_est, t_est



In [ ]:
def TV_MVAR_estimator_MultipleBasisExpansion(data, time, BasisLib, LagOrder):
    """
    Estimate time-varying multivariate autoregressive (TV-MVAR) coefficients using multiple basis expansion.

    This function models TV-MVAR coefficients as a linear combination of predefined
    basis functions. The coefficients for each basis function are estimated using
    least squares, and the time-varying MVAR matrices are reconstructed for each
    time point.

    Parameters
    ----------
    data : ndarray, shape (m, n)
        Multivariate time series data, where `m` is the number of variables (channels)
        and `n` is the number of time points.
    time : ndarray, shape (n,)
        Time points corresponding to each column of `data`.
    BasisLib : ndarray, shape (V, n)
        Library of basis functions used to model time-varying coefficients, where
        `V` is the number of basis functions and `n` is the number of time points.
    LagOrder : int
        Number of lags to include in the MVAR model.

    Returns
    -------
    A_est : list of ndarray, each shape (m, m, LagOrder)
        Estimated TV-MVAR coefficient matrices for each time point.
    t_est : list of float
        Time points corresponding to each estimated coefficient matrix.
    """


    import numpy as np
    import scipy as sp
    # from sklearn.linear_model import Ridge
    
    m,n = data.shape
    Basis_num = BasisLib.shape[0]

    # ======= Prepare the data for least squares estimation =======
    Y = []; X = []
    for i in range(LagOrder, n):
        Y_i = data[:, i].reshape([-1, 1])
        BasisLib_i = BasisLib[:,i]

        X_i = []
        for k in range(Basis_num):
            BasisK_i = BasisLib_i[k]

            for j in range(LagOrder):
                X_ij = data[:, i-j-1].reshape([-1, 1])
                X_i += [ X_ij*BasisK_i ]

        X_i = np.concatenate(X_i, axis=0)

        Y.append(Y_i)
        X.append(X_i)

    Y = np.concatenate(Y, axis=1)
    X = np.concatenate(X, axis=1)

    XtX_inv = np.linalg.inv(X @ X.T)  # Regularization to avoid singular matrix

    # ======= Solve the least squares problem to find coefficients =======
    # model = Ridge(alpha=1e-4)  # or cross-validate for alpha
    # model.fit(X.T, Y.T)
    # Coefs = model.coef_

    Coefs = np.linalg.lstsq(X.T, Y.T, rcond=None)[0].T
    
    Coefs_byBasis = np.split(Coefs, Basis_num, axis=1)   # split coefficients by basis function

    # ======= Estimate time-varying MVAR coefficients =======
    A_est = []
    t_est = []
    for i in range(LagOrder, n):
        A_est_i = np.zeros([m, m*LagOrder])

        for k in range(Basis_num):
            BasisLib_ki = BasisLib[k, i]
            A_est_i += Coefs_byBasis[k] * BasisLib_ki

        A_est.append(np.reshape(A_est_i, [m,m,LagOrder], order='F'))
        t_est.append(time[i])


    return A_est, t_est




In [ ]:
def basis_Legendre(t, degree):
    """
    Generate a Legendre polynomial basis function.

    Parameters
    ----------
    t : ndarray, shape (n,)
        Time points at which to evaluate the basis function.
    degree : int
        Degree of the Legendre polynomial.

    Returns
    -------
    basis : ndarray, shape (n,)
        Evaluated Legendre polynomial at the given time points, rescaled to [-1, 1].
    """
    import numpy as np
    t_rescaled = 2 * (t - t.min()) / (t.max() - t.min()) - 1
    basis_func = np.polynomial.legendre.Legendre.basis(deg=degree)
    basis = basis_func(t_rescaled)
    return basis


def basis_Walsh(t, k):
    """
    Generate the k-th Walsh function.

    Parameters
    ----------
    t : ndarray, shape (n,)
        Time points at which to evaluate the Walsh function.
    k : int
        Order of the Walsh function.

    Returns
    -------
    basis : ndarray, shape (n,)
        Evaluated Walsh function at the given time points, scaled to [0, 1].
    """
    import numpy as np
    import scipy as sp

    def sequency_order(H):
        return sorted(range(H.shape[0]), key=lambda i: np.sum(H[i][1:] != H[i][:-1]))

    t_rescaled = (t - t.min()) / (t.max() - t.min())
    n = int(np.ceil(np.log2(np.max([k+1, 1]))))
    m = 2**n
    H = sp.linalg.hadamard(m)
    order = sequency_order(H)
    basis = H[order[k]]
    step_idx = np.floor(t_rescaled * m).astype(int)
    step_idx = np.clip(step_idx, 0, m-1)
    return basis[step_idx]


def basis_Chebyshev(t, degree):
    """
    Generate a Chebyshev polynomial basis function.

    Parameters
    ----------
    t : ndarray, shape (n,)
        Time points at which to evaluate the Chebyshev polynomial.
    degree : int
        Degree of the Chebyshev polynomial.

    Returns
    -------
    basis : ndarray, shape (n,)
        Evaluated Chebyshev polynomial at the given time points, rescaled to [-1, 1].
    """
    import numpy as np
    t_rescaled = 2 * (t - t.min()) / (t.max() - t.min()) - 1
    basis = np.polynomial.chebyshev.chebvander(t_rescaled, degree)[:, degree]
    return basis


def basis_Fourier_Sin(t, order):
    """
    Generate a Fourier sine basis function.

    Parameters
    ----------
    t : ndarray, shape (n,)
        Time points at which to evaluate the sine basis function.
    order : int
        Order of the sine function.

    Returns
    -------
    basis : ndarray, shape (n,)
        Evaluated Fourier sine function at the given time points, scaled to [0, 1].
    """
    import numpy as np
    t_rescaled = (t - t.min()) / (t.max() - t.min())
    basis = np.sin(2 * np.pi * (order + 1) * t_rescaled)
    return basis


def basis_Fourier_Cos(t, order):
    """
    Generate a Fourier cosine basis function.

    Parameters
    ----------
    t : ndarray, shape (n,)
        Time points at which to evaluate the cosine basis function.
    order : int
        Order of the cosine function.

    Returns
    -------
    basis : ndarray, shape (n,)
        Evaluated Fourier cosine function at the given time points, scaled to [0, 1].
    """
    import numpy as np
    t_rescaled = (t - t.min()) / (t.max() - t.min())
    basis = np.cos(2 * np.pi * (order + 1) * t_rescaled)
    return basis


def basis_Wavelet_Morlet(t, scale=1, shift=0, w0=5.0):
    """
    Generate a Morlet wavelet basis function.

    Parameters
    ----------
    t : ndarray, shape (n,)
        Time points at which to evaluate the Morlet wavelet.
    scale : float, optional
        Scale of the wavelet. Default is 1.
    shift : float, optional
        Shift of the wavelet along time. Default is 0.
    w0 : float, optional
        Central frequency of the Morlet wavelet. Default is 5.0.

    Returns
    -------
    Psi : ndarray, shape (n,)
        Evaluated Morlet wavelet at the given time points.
    """
    import numpy as np
    t_rescaled = (t - shift) / scale
    norm = np.pi**(-0.25) / np.sqrt(scale)
    gaussian = np.exp(-0.5 * t_rescaled**2)
    sinusoid = np.cos(w0 * t_rescaled)
    Psi = norm * gaussian * sinusoid
    return Psi


def basis_Wavelet_Ricker(t, scale=1, shift=0):
    """
    Generate a Ricker (Mexican Hat) wavelet basis function.

    Parameters
    ----------
    t : ndarray, shape (n,)
        Time points at which to evaluate the Ricker wavelet.
    scale : float, optional
        Scale of the wavelet. Default is 1.
    shift : float, optional
        Shift of the wavelet along time. Default is 0.

    Returns
    -------
    Psi : ndarray, shape (n,)
        Evaluated Ricker wavelet at the given time points.
    """
    import numpy as np
    t_rescaled = (t - shift) / scale
    norm = 2 / (np.sqrt(3 * scale) * np.pi**0.25)
    Psi = norm * (1 - t_rescaled**2) * np.exp(-0.5 * t_rescaled**2)
    return Psi


In [ ]:
def plot_TV_matrix(A, t, pValues=None, shareAxes=None):
    """
    Plot time-varying MVAR or similar matrices across all variables and lags.

    This function visualizes each element of a time-varying matrix A over time.
    Optionally, it highlights significant entries based on p-values. Multiple
    lags are plotted in separate figures, and axis sharing can be customized.

    Parameters
    ----------
    A : list of ndarray, each shape (m, m, lags)
        Time-varying matrices to plot. Each element in the list corresponds to a
        specific time point. If 2D matrices are provided, they are assumed to have
        a single lag and reshaped automatically.
    t : list or ndarray, shape (n,)
        Time points corresponding to each A matrix.
    pValues : list of ndarray, optional
        Corresponding p-values for each element in A. Same structure as A.
        Significant values (p < 0.03) are highlighted as red dots on the plots.
    shareAxes : str or None, optional
        Axis sharing option for subplots:
        - 'x': share x-axis
        - 'y': share y-axis
        - 'both': share both axes
        - None: no axis sharing (default)

    Returns
    -------
    None
        Displays one figure per lag, with subplots for each matrix element.
        Significant values are overlaid as red scatter points when `pValues` is provided.
    """

    import numpy as np
    import matplotlib.pyplot as plt

    pValue_critical = 0.03

    time_points = len(t)
    for i in range(time_points):
        if A[i].ndim == 2: A[i] = A[i][:, :, np.newaxis]

    m, _, Lag = A[0].shape

    for p in range(Lag):
        if shareAxes is None:
            fig, ax = plt.subplots(m, m, figsize=(2*m, 2*m), sharex=False, sharey=False)
        elif shareAxes == 'x':
            fig, ax = plt.subplots(m, m, figsize=(2*m, 2*m), sharex=True, sharey=False)
        elif shareAxes == 'y':
            fig, ax = plt.subplots(m, m, figsize=(2*m, 2*m), sharex=False, sharey=True)
        elif shareAxes == 'both':
            fig, ax = plt.subplots(m, m, figsize=(2*m, 2*m), sharex=True, sharey=True)

        for i in range(m):
            for j in range(m):
            # for j in range(i+1):
                A_arrays = [A[w][i, j, p] for w in range(time_points)]
                ax[i, j].plot(t, A_arrays)
                ax[i, j].grid()
                # ax[i, j].set_title(f"A[{i+1}, {j+1}]")

                if pValues is not None:
                    w_plot = [t[w] for w in range(time_points) if pValues[w][i,j,p]<pValue_critical]
                    pValue_plot = [A[w][i,j,p] for w in range(time_points) if pValues[w][i,j,p]<pValue_critical]
                    ax[i, j].scatter(w_plot, pValue_plot, color='r', alpha=0.2, label='p-values')
                    # ax[i, j].legend()

        fig.suptitle(f"A[lag={p+1}]")
        if pValues is not None:
            fig.suptitle(f"A[lag={p+1}] (red dots: p-values < 0.03)")

        # plt.subplots_adjust(wspace=0, hspace=0)
        plt.tight_layout()
        plt.show()

